In [ ]:
# -*- coding: utf-8
# Repórter Brasil (https://ruralometro2022.reporterbrasil.org.br/)
# Reinaldo Chaves (@paidatocandeira)
# # Procura empresas de deputados federais inscritas em autos de infração do ICMBio

In [1]:
import dbf
import pandas as pd
from dbfread import DBF
from simpledbf import Dbf5
import unidecode

PyTables is not installed. No support for HDF output.
SQLalchemy is not installed. No support for SQL output.


In [2]:
def f(str):
    try:
        return (unidecode.unidecode(str))
    except:
        return (str)

In [3]:
dbf = Dbf5('originais/Autos_Infracao_Eletronicos_ICMBio_Atualizado_12072021.dbf', codec='ISO8859-1')
autos_icmbio = dbf.to_dataframe()
autos_icmbio = autos_icmbio[pd.notnull(autos_icmbio['cpf_cnpj'])].copy()
def limpeza1(linha):
    documento = str(linha["cpf_cnpj"])
    documento = documento.strip()
    documento = documento.replace('.', '')
    documento = documento.replace('-', '')
    documento = documento.replace('/', '')
    return str(documento)
autos_icmbio["cpf_cnpj"] = autos_icmbio.apply(limpeza1, axis=1)
# Preenche zeros a esquerda nos documentos
def compara1(linha):
    documento = str(linha["cpf_cnpj"])
    
    if len(documento) < 14 and len(documento) >= 12:
        conta = 14 - (len(documento))
        zero = '0'
        zeros = zero * conta
        alterado = zeros + documento
    elif len(documento) == 14:
        alterado = documento
    elif len(documento) < 11:
        conta = 11 - (len(documento))
        zero = '0'
        zeros = zero * conta
        alterado = zeros + documento
    elif len(documento) == 11:
        alterado = documento
    
    return alterado
autos_icmbio["cpf_cnpj"] = autos_icmbio.apply(compara1, axis=1)
def limpeza2(linha):
    documento = str(linha["cpf_cnpj"])
    if len(documento) > 11:
        return documento[:8]
    else:
        return "documento_cpf"
    
autos_icmbio['cnpj_raiz'] = autos_icmbio.apply(limpeza2, axis=1)

autos_icmbio["autuado_semacentos"] = autos_icmbio["autuado"].str.upper()
autos_icmbio["autuado_semacentos"] = autos_icmbio["autuado_semacentos"].apply(f)

icmbio_pf = autos_icmbio[(autos_icmbio['cnpj_raiz'] == 'documento_cpf')].copy()
icmbio_pj = autos_icmbio[(autos_icmbio['cnpj_raiz'] != 'documento_cpf')].copy()

In [4]:
kwargs = {'sep': ',', 'dtype': str, 'encoding': 'utf-8'}
empresas_politicos = pd.read_csv("resultados/empresas_deputados/empresas_deputados_fev_2022.csv", **kwargs)

In [5]:
kwargs = {'sep': ',', 'dtype': str, 'encoding': 'utf-8'}
empresas_politicos_socios = pd.read_csv("resultados/empresas_deputados/todos_socios_empresas_deputados_fev_2022.csv", **kwargs)

In [6]:
# Procura empresas de deputados em autos do ICMBIO

In [7]:
empresas_politicos.columns

Index(['nome', 'nome_urna', 'politico_id', 'cpf_mascarado', 'cnpj',
       'cnpj_basico_x', 'nome_socio', 'cnpj_cpf_socio', 'razao_social',
       'nome_fantasia', 'situacao_cadastral', 'cnae_fiscal',
       'cnae_fiscal_secundaria', 'uf', 'municipio'],
      dtype='object')

In [8]:
icmbio_pj.columns

Index(['vw_num_aut', 'numero_ai', 'status', 'tipo', 'embargo', 'autuado',
       'cpf_cnpj', 'valor_mult', 'desc_infra', 'tipo_inf', 'data', 'nome_uc',
       'cnuc', 'municipio', 'cod_muni', 'uf', 'operacao', 'orgao_emis',
       'agente', 'enquadrame', 'cod_acao', 'termo_emb', 'obs', 'erro_auto',
       'nr_process', 'julgamento', 'cnpj_raiz', 'autuado_semacentos'],
      dtype='object')

In [9]:
icmbio_pj[['autuado_semacentos', 'cpf_cnpj']]

,autuado_semacentos,cpf_cnpj
1,FRANCISCO ALCENIR RODRIGUES DE SOUZA,31753302000164
2,CIDADE JARDIM COMERCIO DE MADEIRAS LTDA,22327400000149
15,LINHAS DE MACAPA! TANSMISDORAS DE ENERGIA LTDA,10234027000100
19,CERN CONSULT E EMPREE REC NAT LTDA,26026799000189
34,RED BULL DO BRASIL LTDA.,02946761000166
...,...,...
1937,MILGRAN INDUSTRIA E COMARCIO DE GRANITOS LTDA.,07268379000164
1975,VAC/NIA MARIA RAMALHO NUNES,19318769000180
1987,PARTNERS PARTICIPAAAES E EMPREENDIMENTOS IMOBI...,08185728000147
2028,F. CAMPOS PINTO,34032585000143


In [10]:
empresas_icmbio = pd.merge(empresas_politicos.drop_duplicates('cnpj_basico_x'), icmbio_pj.drop_duplicates('cnpj_raiz'), left_on='cnpj_basico_x', right_on='cnpj_raiz', how = 'left')

In [11]:
conta = empresas_icmbio[pd.notnull(empresas_icmbio['cpf_cnpj'])].copy()
conta.shape

(0, 43)

In [33]:
# Procura sócio de empresas de deputados em autos do ICMBIO

In [34]:
empresas_politicos_socios.columns

Index(['cnpj', 'cnpj_basico_x', 'cnpj_basico', 'identificador_de_socio',
       'nome_socio', 'cnpj_cpf_socio', 'qualificacao_socio', 'pais',
       'representante_legal', 'nome_representante'],
      dtype='object')

In [35]:
icmbio_pf.columns

Index(['vw_num_aut', 'numero_ai', 'status', 'tipo', 'embargo', 'autuado',
       'cpf_cnpj', 'valor_mult', 'desc_infra', 'tipo_inf', 'data', 'nome_uc',
       'cnuc', 'municipio', 'cod_muni', 'uf', 'operacao', 'orgao_emis',
       'agente', 'enquadrame', 'cod_acao', 'termo_emb', 'obs', 'erro_auto',
       'nr_process', 'julgamento', 'cnpj_raiz', 'autuado_semacentos'],
      dtype='object')

In [36]:
icmbio_pf['cpf_mascarado']= '***'+icmbio_pf['cpf_cnpj'].str[3:9]+'**'

In [37]:
icmbio_pf[['autuado_semacentos', 'cpf_mascarado']]

,autuado_semacentos,cpf_mascarado
0,LUIZ OSCAR MARQUES FERREIRA,***893881**
3,LUIZ FELIPE GRAVA DO VAL NASCIMENTO,***137988**
4,VILSON PREDOLIM,***018528**
5,MEYRIANE DE SOUSA RODRIGUES,***609953**
6,LUZINETE LIMA ASSUNCAO,***789163**
...,...,...
2033,OSMAR LEANDRO DA SILVA,***901482**
2034,CLODOMIR CEOLATTO,***846981**
2035,INACIO LIBER DA CRUZ,***128262**
2036,LUIZ OTA!VIO AFFONSO CHRISTO,***133917**


In [38]:
socios_icmbio = pd.merge(empresas_politicos_socios.drop_duplicates('cnpj_cpf_socio'), icmbio_pf.drop_duplicates('cpf_cnpj'), left_on=['cnpj_cpf_socio', 'nome_socio'], right_on=['cpf_mascarado','autuado_semacentos'], how = 'left')

In [39]:
conta = socios_icmbio[pd.notnull(socios_icmbio['cpf_cnpj'])].copy()
conta.shape

(0, 39)